In [1]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import datetime as dt
import csv

C:\Users\guilherme.vicari_dot\AppData\Local\Temp\ipykernel_7152\1442181265.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
#hoje = dt.datetime.today().strftime('%Y-%m-%d')
hoje = '2024-04-08'

In [3]:
df_basetotal = pd.read_excel('input_sem_exclusao08042024_teste.xlsx', sheet_name='base_total')
df_recuperi = pd.read_excel('Base_camila_0204.xlsx', header=1)
df_acaocontrole = pd.read_csv('base_grupo_acao_controle.csv')

In [4]:
df_basetotal.shape

(57483, 63)

In [5]:
df_recuperi.rename(columns={'Número do título': 'loan_id'}, inplace=True)
df_recuperi['flag_protesto'] = 1

In [6]:
df_recuperi = df_recuperi.loc[:,['loan_id','flag_protesto']]

In [7]:
df = pd.merge(df_basetotal,df_recuperi, how='left', on='loan_id')

In [8]:
##df['flag_protesto'].fillna(0, inplace=True)
df.fillna({'flag_protesto': 0}, inplace=True)
df['flag_protesto'] = df['flag_protesto'].astype(int)

In [9]:
df.groupby('flag_protesto')['loan_id'].count()

flag_protesto
0    48498
1     8985
Name: loan_id, dtype: int64

In [10]:
df['flag_parcela'] = ''
df['regiao'] = ''
df['custo'] = ''
df['grupo_controle'] = ''
df['acao_controle'] = ''
df['retorno'] = ''

In [11]:
df[['loan_id','valor_parcela','flag_parcela']].head(5)

,loan_id,valor_parcela,flag_parcela
0,23488783,179.85,
1,19853726,422.48,
2,23896506,535.04,
3,22407538,133.24,
4,23901789,280.35,


In [12]:
for indice, linha in df.iterrows():
    if linha['valor_parcela'] > 300:
        df.at[indice, 'flag_parcela'] = '>300'
    else:
        df.at[indice, 'flag_parcela'] = '=<300'

In [13]:
df[['loan_id','valor_parcela','flag_parcela']].head(5)

,loan_id,valor_parcela,flag_parcela
0,23488783,179.85,=<300
1,19853726,422.48,>300
2,23896506,535.04,>300
3,22407538,133.24,=<300
4,23901789,280.35,=<300


In [14]:
df['regiao'] = df['address_state']
#df['regiao'].fillna('na', inplace=True)
df.fillna({'regiao': 0}, inplace=True)

In [15]:
df.groupby('regiao')['loan_id'].count()

regiao
0        10
AC      192
AL      353
AM      847
AP      204
BA     1994
CE     1073
DF     1655
ES     1126
GO     3068
MA      792
MG     5246
MS     1175
MT     1364
PA     1895
PB      535
PE     1051
PI      538
PR     3694
RJ     4141
RN      534
RO      539
RR      166
RS     3444
SC     2793
SE      273
SP    18331
TO      450
Name: loan_id, dtype: int64

In [16]:
for indice, linha in df.iterrows():
    if linha['regiao'] == 'SP':
        df.at[indice, 'custo'] = 3.19
    else:
        df.at[indice, 'custo'] = 9.14

In [17]:
df[['loan_id','valor_parcela','regiao','custo']].head(5)

,loan_id,valor_parcela,regiao,custo
0,23488783,179.85,SE,9.14
1,19853726,422.48,SP,3.19
2,23896506,535.04,PA,9.14
3,22407538,133.24,CE,9.14
4,23901789,280.35,RJ,9.14


In [18]:
df_acaocontrole.shape

(12942, 11)

In [19]:
df_grupocontrole = df_acaocontrole.query("data_retorno > @hoje")

In [20]:
df_grupocontrole.shape

(4069, 11)

In [23]:
df_grupocontrole['data_inicio_protesto'] = pd.to_datetime(df_grupocontrole['data_inicio_protesto'])

df_grupocontrole['diferenca_dias'] = (dt.datetime.today() - df_grupocontrole['data_inicio_protesto']).dt.days

df_grupocontrole1 = df_grupocontrole.sort_values(by='diferenca_dias', ascending=False)

df_grupocontrole1 = df_grupocontrole1.drop_duplicates(subset='loan_id')

df_grupocontrole1.drop(columns='diferenca_dias', inplace=True)

In [ ]:
df_grupocontrole1['flag_controle'] = '1'

In [ ]:
df_grupocontrole1 = df_grupocontrole1.loc[:,['loan_id','flag_controle','grupo']]

In [ ]:
df.shape

In [ ]:
df = pd.merge(df,df_grupocontrole1,how='left', on='loan_id')

In [ ]:
df.shape

In [ ]:
#df['flag_controle'].fillna(0, inplace=True)
df.fillna({'flag_controle': 0}, inplace=True)
df['flag_controle'] = df['flag_controle'].astype(int)

In [ ]:
df.groupby('flag_controle')['loan_id'].count()

In [ ]:
df['grupo_controle'] = df['flag_controle']
df['acao_controle'] = df['grupo']

In [ ]:
df = df.drop(['flag_controle','grupo'], axis=1)

In [ ]:
df_retorno = df_acaocontrole.query("data_retorno <= @hoje")

In [ ]:
df.shape

In [ ]:
df_retorno['data_inicio_protesto'] = pd.to_datetime(df_retorno['data_inicio_protesto'])

df_retorno['diferenca_dias'] = (dt.datetime.today() - df_retorno['data_inicio_protesto']).dt.days

df_retorno = df_retorno.sort_values(by='diferenca_dias', ascending=False)

df_retorno = df_retorno.drop_duplicates(subset='loan_id')

df_retorno.drop(columns='diferenca_dias', inplace=True)

In [ ]:
df_retorno.shape

In [ ]:
df_retorno = df_retorno.loc[:,['loan_id','data_retorno','semana']]

In [ ]:
df_retorno.head(3)

In [ ]:
df_retorno = df_retorno[df_retorno['data_retorno'] <= hoje]

In [ ]:
df_retorno.shape

In [ ]:
df = pd.merge(df,df_retorno, how='left', on='loan_id')

In [ ]:
df.shape

In [ ]:
df['retorno'] = df['semana']
df['retorno'].fillna(0, inplace=True)

In [ ]:
df = df.drop(['data_retorno','semana'], axis=1)

In [ ]:
atraso = ['3 - 60a90', '4 - 90a120', '5 - 120a150']
df_final = df[df['late_category_expandida_noverde_atual'].isin(atraso)]

In [ ]:
df_final.shape

In [ ]:
flag_volumetria = df_final.pivot_table(index='late_category_expandida_noverde_atual', 
                             columns='flag_protesto', 
                             values='loan_id', 
                             aggfunc='count', 
                             margins=True, 
                             margins_name='Total')

print(flag_volumetria)

In [ ]:
acao_controle = df_final.query("grupo_controle ==1")

In [ ]:
flag_acaocontrole = acao_controle.pivot_table(index='late_category_expandida_noverde_atual', 
                             columns='acao_controle', 
                             values='loan_id', 
                             aggfunc='count', 
                             margins=True, 
                             margins_name='Total')

print(flag_acaocontrole)

In [ ]:
protesto = df_final.query("grupo_controle ==0")

In [ ]:
flag_protesto = protesto.pivot_table(index='late_category_expandida_noverde_atual', 
                             columns='flag_protesto', 
                             values='loan_id', 
                             aggfunc='count', 
                             margins=True, 
                             margins_name='Total')

print(flag_protesto)

In [ ]:
acordoavencer = df_final.query("grupo_controle ==0 & flag_protesto==0")

In [ ]:
flag_acordo = acordoavencer.pivot_table(index='late_category_expandida_noverde_atual', 
                             columns='acordo_a_vencer', 
                             values='loan_id', 
                             aggfunc='count', 
                             margins=True, 
                             margins_name='Total')

print(flag_acordo)

In [ ]:
df_final.shape